In [ ]:
import pandas as pd
from tqdm import tqdm
from vcae_mnist.config import config
from vcae_mnist.experiment import run_experiment

results = []

for seed in tqdm(range(1), desc="Running experiments"):
    result = run_experiment(seed, config)
    results.append(result)


df_results = pd.DataFrame(results)
df_results.to_csv("experiment_results.csv", index=False)

Running experiments:   0%|          | 0/1 [00:00<?, ?it/s]Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('GRID A100D-16C') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=10` reached.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, usin

In [12]:
import glob
import os

import pandas as pd

# Replace with your folder path
folder = os.getcwd()

# List all matching CSV files
csv_files = glob.glob(os.path.join(folder, "results_*.csv"))

# Read and concatenate all into a single DataFrame
df_all = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)

In [55]:
df_long = (
    df_all.melt(id_vars="seed", var_name="variable", value_name="value")
    .assign(
        metric=lambda x: x["variable"].str.extract(r"^([a-zA-Z]+)"),
        method=lambda x: x["variable"].str.extract(r"_(.*)$").fillna(""),
    )
    .drop(columns=["variable"])
    .pivot(index=["seed", "method"], columns="metric", values="value")
    .reset_index()
    .drop(columns=["seed"])
)

In [59]:
import numpy as np


def mean_ci(x, digits: int = 3) -> str:
    x = np.asarray(x, dtype=np.float64)
    n = x.size
    if n == 0:
        return "nan ± nan"
    mean = np.mean(x)
    se = np.std(x, ddof=1) / np.sqrt(n)
    ci_half_width = 1.96 * se
    return f"{round(mean, digits)} ± {round(ci_half_width, digits)}"


summary = df_long.groupby("method").agg(mean_ci)
summary.columns.name = None
summary

,fid,loglik,mmd,mse
method,,,,
initial,6.259 ± 0.099,-17.179 ± 1.131,0.514 ± 0.027,0.054 ± 0.001
refit,6.62 ± 0.197,-8.098 ± 1.199,0.492 ± 0.03,0.052 ± 0.001
